<a href="https://colab.research.google.com/github/hajar-hajji/2A-project-MinesNancy-Loria/blob/main/MedicalAbstracts_MultilabelClassification/MedicalAbstracts_multilabel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### This work was done by me as a part my academic project
##### *problem description and dataset used,* [*clic*](https://huggingface.co/datasets/owaiskha9654/PubMed_MultiLabel_Text_Classification_Dataset_MeSH)

#### Loading our dataset

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import numpy as np

In [3]:
mydataset = load_dataset("owaiskha9654/PubMed_MultiLabel_Text_Classification_Dataset_MeSH")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/owaiskha9654___csv/owaiskha9654--PubMed_MultiLabel_Text_Classification_Dataset_MeSH-133881ae76bc4363/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
#features+len
mydataset 

DatasetDict({
    train: Dataset({
        features: ['Title', 'abstractText', 'meshMajor', 'pmid', 'meshid', 'meshroot', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z'],
        num_rows: 50000
    })
})

In [5]:
mydataset["train"][:2]

{'Title': ['Expression of p53 and coexistence of HPV in premalignant lesions and in cervical cancer.',
  'Vitamin D status in pregnant Indian women across trimesters and different seasons and its correlation with neonatal serum 25-hydroxyvitamin D levels.'],
 'abstractText': ['Fifty-four paraffin embedded tissue sections from patients with dysplasia (21 cases) and with cervical cancer (33 cases) were analysed. HPV was detected and identified in two stages. Firstly, using mixed starters, chosen genomic DNA sequences were amplified; secondly the material thus obtained was analyzed by hybridization method using oligonucleotyde 31-P labelled probe. HPVs of type 6, 11, 16, 18, 33 were identified. The p-53 expression was assayed by immunohistochemical method. HPV infection was often associated with dysplasia and cervical cancer. In cervical cancer mainly HPV 16 and 18 with high oncogenic potential were found. The p-53 was present rarely, and in minute quantities. No correlation was observed 

In [6]:
mydataset["train"][3]
for key in mydataset["train"][3]:
    print(key)

Title
abstractText
meshMajor
pmid
meshid
meshroot
A
B
C
D
E
F
G
H
I
J
L
M
N
Z


In [7]:
###Combine title and abstract
#mydataset = mydataset.map(lambda example: {"fullText": example["Title"] + "\n" + example["abstractText"]})
def combine_text(example):
  title=example["Title"] or ""
  abstract=example["abstractText"] or ""
  return {"fullText": title + "\n" + abstract}
#apply to every example
mydataset=mydataset.map(combine_text)

#only keep columns that we will need
columns_to_remove=["Title","abstractText","meshMajor","pmid","meshid","meshroot"]
mydataset=mydataset.remove_columns(columns_to_remove)
mydataset["train"][:2]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'A': [0, 0],
 'B': [1, 1],
 'C': [1, 1],
 'D': [1, 1],
 'E': [1, 1],
 'F': [0, 1],
 'G': [0, 1],
 'H': [1, 0],
 'I': [0, 1],
 'J': [0, 1],
 'L': [0, 0],
 'M': [0, 1],
 'N': [0, 1],
 'Z': [0, 1],
 'fullText': ['Expression of p53 and coexistence of HPV in premalignant lesions and in cervical cancer.\nFifty-four paraffin embedded tissue sections from patients with dysplasia (21 cases) and with cervical cancer (33 cases) were analysed. HPV was detected and identified in two stages. Firstly, using mixed starters, chosen genomic DNA sequences were amplified; secondly the material thus obtained was analyzed by hybridization method using oligonucleotyde 31-P labelled probe. HPVs of type 6, 11, 16, 18, 33 were identified. The p-53 expression was assayed by immunohistochemical method. HPV infection was often associated with dysplasia and cervical cancer. In cervical cancer mainly HPV 16 and 18 with high oncogenic potential were found. The p-53 was present rarely, and in minute quantities. No co

In [8]:
tags=[tag for tag in mydataset["train"][0] if tag!="fullText"]
print(tags)
print(len(tags))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']
14


In [9]:
###Meaning of each label will be stocked in a dictionary
main_ref={"A":"Anatomy","B":"Organisms","C":"Diseases","D":"Chemicals and Drugs",\
          "E":"Analytical, Diagnostic and Therapeutic Techniques, and Equipment","F":"Psychiatry and Psychology",\
          "G":"Phenomena and Processes","H":"Disciplines and Occupations","I":"Anthropology, Education, Sociology, and Social Phenomena",
          "J":"Technology, Industry, and Agriculture","L":"Information Science","M":"Named Groups","N":"Health Care",\
          "Z":"Geographicals"}

In [10]:
main_ref #info from source

{'A': 'Anatomy',
 'B': 'Organisms',
 'C': 'Diseases',
 'D': 'Chemicals and Drugs',
 'E': 'Analytical, Diagnostic and Therapeutic Techniques, and Equipment',
 'F': 'Psychiatry and Psychology',
 'G': 'Phenomena and Processes',
 'H': 'Disciplines and Occupations',
 'I': 'Anthropology, Education, Sociology, and Social Phenomena',
 'J': 'Technology, Industry, and Agriculture',
 'L': 'Information Science',
 'M': 'Named Groups',
 'N': 'Health Care',
 'Z': 'Geographicals'}

In [11]:
#dictionary to get index of each label
get_index=dict()
i=0
for tag in tags:
    get_index[tag]=i
    i+=1
get_index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'L': 10,
 'M': 11,
 'N': 12,
 'Z': 13}

In [12]:
#get tag from index
get_tag={i:tag for i,tag in enumerate(tags)}
get_tag

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'L',
 11: 'M',
 12: 'N',
 13: 'Z'}

####Preprocessing dataset (encoding texts &labels)

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.6 MB/s eta 0:00:00


In [14]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")

In [15]:
#i'll set max_length to 450
mytextt=mydataset["train"]["fullText"][:3]
encoder=tokenizer(mytextt,
                is_split_into_words=False, #phrases non tokenisées
                padding="max_length", truncation=True,      #assure all inputs have same len
                max_length=450)

In [16]:
sample=mydataset["train"][:3]
dic_labels={key:sample[key] for key in sample.keys() if key in tags}
dic_labels

{'A': [0, 0, 1],
 'B': [1, 1, 1],
 'C': [1, 1, 0],
 'D': [1, 1, 1],
 'E': [1, 1, 1],
 'F': [0, 1, 0],
 'G': [0, 1, 1],
 'H': [1, 0, 0],
 'I': [0, 1, 0],
 'J': [0, 1, 0],
 'L': [0, 0, 1],
 'M': [0, 1, 0],
 'N': [0, 1, 0],
 'Z': [0, 1, 0]}

In [17]:
columns=len(tags)
lines=len(sample["A"])
mymatrix=np.zeros((lines,columns))
mymatrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
for i,k in enumerate(dic_labels.keys()):
  mymatrix[:,i]=dic_labels[k]
mymatrix

array([[0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.]])

In [19]:
for i,k in enumerate(dic_labels.keys()):
  mymatrix[:,i]=dic_labels[k]

In [20]:
encoder["labels"]=mymatrix.astype(int).tolist()

In [21]:
encoder.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [22]:
del encoder["token_type_ids"]

In [23]:
encoder.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [24]:
encoder

{'input_ids': [[101, 3670, 1997, 1052, 22275, 1998, 24873, 9048, 16173, 3401, 1997, 6522, 2615, 1999, 26563, 11475, 27881, 22520, 1998, 1999, 28711, 4456, 1012, 5595, 1011, 2176, 11498, 15379, 11157, 8153, 5433, 2013, 5022, 2007, 1040, 7274, 24759, 15396, 1006, 2538, 3572, 1007, 1998, 2007, 28711, 4456, 1006, 3943, 3572, 1007, 2020, 20302, 23274, 2094, 1012, 6522, 2615, 2001, 11156, 1998, 4453, 1999, 2048, 5711, 1012, 15847, 1010, 2478, 3816, 29400, 1010, 4217, 8991, 22026, 6064, 10071, 2020, 26986, 1025, 16378, 1996, 3430, 2947, 4663, 2001, 16578, 2011, 8893, 3989, 4118, 2478, 19330, 14031, 11231, 14321, 4140, 18124, 2861, 1011, 1052, 18251, 15113, 1012, 6522, 15088, 1997, 2828, 1020, 1010, 2340, 1010, 2385, 1010, 2324, 1010, 3943, 2020, 4453, 1012, 1996, 1052, 1011, 5187, 3670, 2001, 4632, 4710, 2098, 2011, 10047, 23041, 11631, 20483, 15869, 4118, 1012, 6522, 2615, 8985, 2001, 2411, 3378, 2007, 1040, 7274, 24759, 15396, 1998, 28711, 4456, 1012, 1999, 28711, 4456, 3701, 6522, 2615, 23

In [ ]:
#max len in train dataset, just to get an idea about the magnitude of sentences in the dataset
# max_length=len(mydataset["train"]["fullText"][0])
# max_sentence=""
# for i,sentence in enumerate((mydataset["train"]["fullText"])):
#     if len(mydataset["train"]["fullText"][i])>max_length:
#         max_length=len(mydataset["train"]["fullText"][i])
#         max_sentence=sentence
# print(max_sentence)
# print(max_length)

# """ps. do not rerun this cell !!!!!! it takes so long to run and it's of no use for the study as i already chose max_len ;)
# """

In [25]:
###now let's define a function that will apply these steps for a certain dataset
def preprocessing(sample):
  #####Encode all texts###############
  #i'll set max_length to 512
  mytext=sample["fullText"]
  encoder=tokenizer(mytext,
                is_split_into_words=False, #phrases non tokenisées
                padding="max_length", truncation=True,      #assure all inputs have same len
                max_length=512)
  #returns dictionary : {"input_ids"}
  
  #####Encode all labels###############
  #create dictionary that contains labels 
  dic_labels={key:sample[key] for key in sample.keys() if key in tags}
  #create an empty matrix
  columns=len(tags)
  lines=len(sample["A"])
  mymatrix=np.zeros((lines,columns))


  for i,k in enumerate(dic_labels.keys()):
    mymatrix[:,i]=dic_labels[k]
  #that way we'll have a matrix where each line of labels corresponds to a specific text, 1 if label and 0 else...

  #add labels to the main dictionary
  encoder["labels"]=mymatrix.tolist()
  #delete token_type_ids bcs we won't need it
  del encoder["token_type_ids"] # if AutoModelForSequenceClassification model is used, it will ignore the token_type_ids argument anyway.

  return encoder

In [26]:
""" preprocess dataset & delete the initial columns ("fullText" and the remaining labels...)
only encoded values are preserved ()
"""
preprocessed_data=mydataset.map(preprocessing,batched=True,remove_columns=mydataset["train"].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [27]:
type(preprocessed_data)

datasets.dataset_dict.DatasetDict

####Split dataset (train/val)

In [28]:
from sklearn.model_selection import train_test_split

train_ids,val_ids,train_mask,val_mask,train_labels,val_labels=train_test_split(preprocessed_data["train"]["input_ids"],\
                                                                   preprocessed_data["train"]["attention_mask"],\
                                                                   preprocessed_data["train"]["labels"]\
                                                                   ,test_size=.2,random_state=0)

train_data={"input_ids":train_ids,"attention_mask":train_mask,"labels":train_labels}
val_data={"input_ids":val_ids,"attention_mask":val_mask,"labels":val_labels}

In [29]:
###random check :)
decoded_obs=tokenizer.decode(train_data["input_ids"][19], skip_special_tokens=True)
print(decoded_obs)
print(train_data["input_ids"][19])
print(train_data["labels"][19])
tags_19=[get_tag[i] for i in range(len(train_data["labels"][19])) if train_data["labels"][19][i]==1]  #1:tag exists
print(tags_19)
refs=[main_ref[tag] for tag in tags_19]
print(refs)

sustained c - jun - nh2 - kinase activity promotes epithelial - mesenchymal transition, invasion, and survival of breast cancer cells by regulating extracellular signal - regulated kinase activation. the c - jun nh ( 2 ) - terminus kinase ( jnk ) mediates stress - induced apoptosis and the cytotoxic effect of anticancer therapies. paradoxically, recent clinical studies indicate that elevated jnk activity in human breast cancer is associated with poor prognosis. here, we show that overexpression of a constitutively active jnk in human breast cancer cells did not cause apoptosis, but actually induced cell migration and invasion, a morphologic change associated with epithelial - mesenchymal transition ( emt ), expression of mesenchymal - specific markers vimentin and fibronectin, and activity of activator protein transcription factors. supporting this observation, mouse mammary tumor cells that have undergone emt showed upregulated jnk activity, and the emt was reversed by jnk inhibition.

In [30]:
###random check round two :)
decoded_obs=tokenizer.decode(val_data["input_ids"][24], skip_special_tokens=True)
print(decoded_obs)
print(val_data["input_ids"][24])
print(val_data["labels"][24])
tags_24=[get_tag[i] for i,tag in enumerate(val_data["labels"][24]) if tag==1] #1:tag exists
print(tags_24)
refs=[main_ref[tag] for tag in tags_24]
print(refs)

enhanced sorption of polycyclic aromatic hydrocarbons from aqueous solution by modified pine bark. to enhance removal efficiency of natural sorbent with polycyclic aromatic hydrocarbons ( pahs ), single - solute and bi - solute sorption of phenanthrene and pyrene onto raw and modified pine bark were investigated. pine bark was modified using soxhlet extraction, saponification and acid hydrolysis, yielding six bark fractions with different chemical compositions. raw pine bark exhibited high affinities with pahs, and sorption was dominated by partitioning. the relatively nonlinear sorption isotherms of modified bark were attributed to the specific interaction between sorbate and aromatic core of sorbent. comparison with lipid and suberin, lignin was the most powerful sorption medium, but which was almost completely suppressed by coexisting polysaccharide. after consuming polysaccharide by acid hydrolysis, sorption of pine bark fractions was notably increased ( 4 - 17 folds ) ; and sorpti

In [31]:
from datasets import Dataset
import torch

#transform to dataset object
train_data_copy=Dataset.from_dict(train_data)
val_data_copy=Dataset.from_dict(val_data)
print(type(train_data_copy))

# import copy
# train_data_copy2=copy.deepcopy(train_data_copy)

###transform all datasets to tensors before feeding them to the model(ps: directly w/ affecting variables...!!)
train_data_copy.set_format("torch")
print(type(train_data_copy))
val_data_copy.set_format("torch")

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
# from datasets import DatasetDict

# train_data_copyy=DatasetDict(train_data)
# train_data_essai=train_data.copy()
# train_data_copy.set_format("torch")
##""--
# input_ids_train=train_data["input_ids"]
# mask_train=train_data["attention_mask"]
# labels_train=train_data["labels"]
# tensor_train=TensorDataset(torch.tensor(input_ids_train),torch.tensor(mask_train),torch.tensor(labels_train))

#### Introducing the model

In [32]:
from transformers import BertForSequenceClassification
model=BertForSequenceClassification.from_pretrained("bert-base-uncased",problem_type="multi_label_classification",num_labels=len(tags),id2label=get_tag,label2id=get_index)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### Train the model (fast trainer)

In [33]:
from transformers import TrainingArguments, Trainer

epochs=1
lr=1e-5
batch_size=16

arguments = TrainingArguments(output_dir="/content/drive/MyDrive/mymodels/med_classif_abstract",
    evaluation_strategy = "epoch",save_strategy="epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [34]:
#let's see what the model output looks like (initially)
output_1 = model(input_ids=train_data_copy["input_ids"][0].unsqueeze(0),labels=train_data_copy["labels"][0].unsqueeze(0))
output_1

SequenceClassifierOutput(loss=tensor(0.7532, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0321,  0.2688,  0.1807, -0.3697,  0.3948, -0.1747, -0.4591, -0.1441,
          0.2316,  0.4735, -0.1143,  0.5607, -0.8226, -0.3508]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [35]:
print(output_1[0]) #loss
print()
print(output_1[1]) #logits
#ps : not normalized, meaning that they're not necessarily between 0 and 1 and are not interpretable as probabilities...
print()
print(output_1[1].shape)

tensor(0.7532, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([[-0.0321,  0.2688,  0.1807, -0.3697,  0.3948, -0.1747, -0.4591, -0.1441,
          0.2316,  0.4735, -0.1143,  0.5607, -0.8226, -0.3508]],
       grad_fn=<AddmmBackward0>)

torch.Size([1, 14])


In [36]:
probabilities=torch.sigmoid(output_1[1])
probabilities

tensor([[0.4920, 0.5668, 0.5451, 0.4086, 0.5974, 0.4564, 0.3872, 0.4640, 0.5576,
         0.6162, 0.4714, 0.6366, 0.3052, 0.4132]], grad_fn=<SigmoidBackward0>)

In [37]:
print("Initial predictions :")
predictions=[1 if el>0.5 else 0 for el in probabilities.tolist()[0]]
print(predictions)
literal_predictions=[get_tag[i] for i,el in enumerate(predictions) if el==1]
print(literal_predictions)
interpreted_pred=[main_ref[tag] for tag in literal_predictions]
print(interpreted_pred)
print("\n")
print("True labels :")
#real labels
labels=train_data_copy["labels"][0]
print(labels)
lit=[get_tag[i] for i,el in enumerate(labels) if el==1]
print(lit)
ref=[main_ref[tag] for tag in lit]
print(ref)

Initial predictions :
[0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
['B', 'C', 'E', 'I', 'J', 'M']
['Organisms', 'Diseases', 'Analytical, Diagnostic and Therapeutic Techniques, and Equipment', 'Anthropology, Education, Sociology, and Social Phenomena', 'Technology, Industry, and Agriculture', 'Named Groups']


True labels :
tensor([1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.])
['A', 'B', 'E', 'G', 'L', 'N']
['Anatomy', 'Organisms', 'Analytical, Diagnostic and Therapeutic Techniques, and Equipment', 'Phenomena and Processes', 'Information Science', 'Health Care']


In [38]:
output_2 = model(input_ids=train_data_copy["input_ids"][:2],labels=train_data_copy["labels"][:2])
output_2

SequenceClassifierOutput(loss=tensor(0.7798, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0321,  0.2688,  0.1807, -0.3697,  0.3948, -0.1747, -0.4591, -0.1441,
          0.2316,  0.4735, -0.1143,  0.5607, -0.8226, -0.3508],
        [-0.0610,  0.2731,  0.1877, -0.3594,  0.3711, -0.2088, -0.4294, -0.1975,
          0.2379,  0.4534, -0.0987,  0.5458, -0.7977, -0.2637]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [39]:
print(output_2[1].shape) #dim: (batch_size,num_tags)

torch.Size([2, 14])


In [40]:
from sklearn.metrics import precision_recall_fscore_support
from transformers import EvalPrediction

"""
EvalPrediction class from the Hugging Face Transformers library
The EvalPrediction class has two main attributes: predictions and label_ids. 

1) predictions is a tensor of size (batch_size, num_labels) containing the model's output scores for each example in the batch.
2) label_ids is a tensor of the same size containing the actual labels for each example in the batch.
3) ...

source : https://huggingface.co/docs/transformers/internal/trainer_utils
"""

def function_metrics(p: EvalPrediction):  #an instance of EvalPrediction

    logits=p.predictions
    probabilities=torch.sigmoid(torch.tensor(logits)) #turn into probabilities using sigmoid activation function
    predicted_labels=np.where(probabilities>=0.5,1,0) #set 1 if belongs to class 0 otherwise

    true_labels=p.label_ids
    
    #precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average=None)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='micro')
    
    return {
    'precision_micro': precision_micro,
    'recall_micro': recall_micro,
    'f1_micro': f1_micro,
    }


In [ ]:
#fast trainer -> https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model,
    arguments,
    train_dataset=train_data_copy,
    eval_dataset=val_data_copy,
    tokenizer=tokenizer,
    compute_metrics=function_metrics
)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_cached_bytes=2147483648,max_split_size_mb=1024'

In [ ]:
%time trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 109493006


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro
1,0.308500,0.304385,0.856504,0.833767,0.844983


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500
Configuration saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/config.json
Model weights saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/special_tokens_map.json


KeyError: ignored

####Evaluate the model


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro
1,0.308500,0.304385,0.856504,0.833767,0.844983
1,0.308500,0.304385,0.856504,0.833767,0.844983


{'eval_loss': 0.3043845593929291,
 'eval_precision_micro': 0.8565042547843883,
 'eval_recall_micro': 0.8337673231855438,
 'eval_f1_micro': 0.8449828644297442}

####Save the model

In [ ]:
#save tokenizer
tokenizer.save_vocabulary("/content/mymodels/Medical_classif_abstracts_1e")
#save trained model
model.save_pretrained("/content/mymodels/Medical_classif_abstracts_1e")

Configuration saved in /content/mymodels/Medical_classif_abstracts_1e/config.json
Model weights saved in /content/mymodels/Medical_classif_abstracts_1e/pytorch_model.bin


####Continue training on +1 epoch

#####Reloading saved model

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
model_path="/content/drive/MyDrive/mymodels/medical_multiL_abstracts_1e"

NEWtokenizer=BertTokenizer.from_pretrained(model_path)
NEWmodel=BertForSequenceClassification.from_pretrained(model_path)

#####Continue training

In [44]:
trainer = Trainer(
    NEWmodel,
    arguments,
    train_dataset=train_data_copy,
    eval_dataset=val_data_copy,
    tokenizer=NEWtokenizer,
    compute_metrics=function_metrics
)

In [45]:
#train on one more epoch
%time trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 109493006


Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro
1,0.280800,0.279347,0.867547,0.845791,0.856531


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500
Configuration saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/config.json
Model weights saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/mymodels/med_classif_abstract/checkpoint-2500/special_tokens_map.json


KeyError: ignored

In [46]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


Epoch,Training Loss,Validation Loss,Precision Micro,Recall Micro,F1 Micro
1,0.280800,0.279347,0.867547,0.845791,0.856531
1,0.280800,0.279347,0.867547,0.845791,0.856531


{'eval_loss': 0.27934730052948,
 'eval_precision_micro': 0.8675474133295092,
 'eval_recall_micro': 0.8457908809702731,
 'eval_f1_micro': 0.856531011353279}

In [63]:
NEWtokenizer.save_vocabulary("/content/drive/MyDrive/mymodels/medical_multiL_abstracts_2e")
NEWmodel.save_pretrained("/content/drive/MyDrive/mymodels/medical_multiL_abstracts_2e")

Configuration saved in /content/drive/MyDrive/mymodels/medical_multiL_abstracts_2e/config.json
Model weights saved in /content/drive/MyDrive/mymodels/medical_multiL_abstracts_2e/pytorch_model.bin


####Test on new abstracts

#####First example

In [47]:
model=NEWmodel
tokenizer=NEWtokenizer

In [48]:
abstract1="rokinase receptor : a molecular organizer in cellular communication .In a variety of cell types , the glycolipid - anchored urokinase receptor ( uPAR ) is colocalized pericellularly with components of the plasminogen activation system and endocytosis receptors . uPAR is also coexpressed with caveolin and members of the integrin adhesion receptor superfamily . The formation of functional units with these various proteins allows the uPAR to mediate the focused proteolysis required for cell migration and invasion and to contribute both directly and indirectly to cell adhesive processes in a non - proteolytic fashion .This dual activity , together with the initiation of signal transduction pathways by uPAR , is believed to influence cellular behaviour in angiogenesis , inflammation , wound repair and tumor progression / metastasis and open up the way for uPAR - based therapeutic approaches ."

#####Encoding######
encode=tokenizer(abstract1,is_split_into_words=False)


In [ ]:
"""AC with torch.no_grad():
    input_ids = torch.tensor(encode.input_ids).to(device)
    attention_mask = torch.tensor(encode.attention_mask).to(device)
    output = model(input_ids, attention_mask=attention_mask)
"""

{'input_ids': [101, 20996, 4939, 11022, 10769, 1024, 1037, 8382, 19012, 1999, 12562, 4807, 1012, 1999, 1037, 3528, 1997, 3526, 4127, 1010, 1996, 1043, 2135, 25778, 11514, 3593, 1011, 14453, 24471, 23212, 11649, 2063, 10769, 1006, 2039, 2906, 1007, 2003, 8902, 24755, 28931, 2566, 6610, 3363, 7934, 2135, 2007, 6177, 1997, 1996, 20228, 3022, 10020, 23924, 13791, 2291, 1998, 2203, 10085, 22123, 12650, 13833, 1012, 2039, 2906, 2003, 2036, 24873, 2595, 19811, 2007, 5430, 18861, 1998, 2372, 1997, 1996, 20014, 13910, 6657, 4748, 21471, 10769, 24169, 1012, 1996, 4195, 1997, 8360, 3197, 2007, 2122, 2536, 8171, 4473, 1996, 2039, 2906, 2000, 2865, 2618, 1996, 4208, 4013, 2618, 4747, 20960, 3223, 2005, 3526, 9230, 1998, 5274, 1998, 2000, 9002, 2119, 3495, 1998, 17351, 2000, 3526, 4748, 21579, 6194, 1999, 1037, 2512, 1011, 4013, 2618, 4747, 21252, 4827, 1012, 2023, 7037, 4023, 1010, 2362, 2007, 1996, 17890, 1997, 4742, 9099, 16256, 16910, 2011, 2039, 2906, 1010, 2003, 3373, 2000, 3747, 12562, 9164, 

In [51]:
mydevice = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(mydevice)

cuda


In [53]:
#turn everything into pytorch tensors (don't forget to move to cuda...)
enc_tensor={key:torch.tensor(value).to(mydevice) for key,value in encode.items()} #or simply specify in the previous line: return_tensors="pt"

In [54]:
print(enc_tensor["input_ids"].device)

cuda:0


In [55]:
with torch.no_grad(): #optional when doing inference, but it's recommended to do it in order to disable gradient computation to save memory and computation time
  output=model(enc_tensor["input_ids"].unsqueeze(0),attention_mask=enc_tensor["attention_mask"].unsqueeze(0))
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.7043,  3.3786, -1.8283,  4.4831, -0.1790, -3.0194,  2.7721, -2.4975,
         -4.7531, -3.3341, -2.1064, -5.0834, -3.4248, -5.3879]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [56]:
logits=output.logits
probas=torch.sigmoid(logits.cpu())
print(probas)
pred=np.where(probas>=0.5,1,0)
print(pred)

pred_tags=[get_tag[i] for i,tag in enumerate(pred.tolist()[0]) if tag==1]
print(pred_tags)
interpreted_tags=[main_ref[tag] for tag in pred_tags]
print(interpreted_tags)

tensor([[0.8461, 0.9670, 0.1384, 0.9888, 0.4554, 0.0466, 0.9412, 0.0760, 0.0086,
         0.0344, 0.1085, 0.0062, 0.0315, 0.0046]])
[[1 1 0 1 0 0 1 0 0 0 0 0 0 0]]
['A', 'B', 'D', 'G']
['Anatomy', 'Organisms', 'Chemicals and Drugs', 'Phenomena and Processes']


#####Second example

In [58]:
abstract2="Role of the interferon - inducible IFI16 gene in the induction of ICAM - 1 by TNF - alpha . The Interferon - inducible gene IFI16 , a member of the HIN200 family , is activated by oxidative stress and cell density , in addition to Interferons , and it is implicated in the regulation of endothelial cell proliferation and vessel formation in vitro . We have previously shown that IFI16 is required for proinflammatory gene stimulation by IFN - gamma through the NF - kappaB complex . To examine whether IFI16 induction might be extended to other proinflammatory cytokines such as tumor necrosis factor ( TNF ) - alpha , we used the strategy of the RNA interference to knock down IFI16 express i on , and analyze the capability of TNF - alpha to stimulate intercellular adhesion molecule - 1 ( ICAM - 1 or CD54 ) express i on in the absence of functional IFI16 . Our studies demonstrate that IFI16 mediates ICAM - 1 stimulation by TNF - alpha through the NF - kappaB pathway , thus reinforcing the role of the IFI16 molecule in the inflammation process ."

encode2=tokenizer(abstract2,is_split_into_words=False)
enc_tensor2={key:torch.tensor(value).to(mydevice) for key,value in encode2.items()} 

with torch.no_grad():
  output2=model(enc_tensor2["input_ids"].unsqueeze(0),attention_mask=enc_tensor2["attention_mask"].unsqueeze(0))

logits=output2.logits
probas=torch.sigmoid(logits.cpu())
print(probas)
pred=np.where(probas>=0.5,1,0)
print(pred)

pred_tags=[get_tag[i] for i,tag in enumerate(pred.tolist()[0]) if tag==1]
print(pred_tags)
interpreted_tags=[main_ref[tag] for tag in pred_tags]
print(interpreted_tags)

tensor([[0.7674, 0.9700, 0.1592, 0.9937, 0.5032, 0.0169, 0.9591, 0.0516, 0.0065,
         0.0300, 0.0977, 0.0090, 0.0354, 0.0054]])
[[1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
['A', 'B', 'D', 'E', 'G']
['Anatomy', 'Organisms', 'Chemicals and Drugs', 'Analytical, Diagnostic and Therapeutic Techniques, and Equipment', 'Phenomena and Processes']


In [ ]:
###END